In [1]:
import tensorflow as tf
from os import path
from my_libs.tf_graph_saver import ScalerGraphSaver

checkpoint_path = "tf_checkpoints"

def save_checkpoint(step):
    chk_path = "{}/epoch.ckpt".format(checkpoint_path)
    sess = tf.get_default_session()
    saver = tf.train.Saver(max_to_keep=2,)
    saver.save(sess, chk_path)
    
    
def restore_checkpoint():
    chk_path = "{}/epoch.ckpt".format(checkpoint_path)
    sess = tf.get_default_session()
    if path.exists(chk_path+".index"):
        saver = tf.train.Saver(max_to_keep=2,)
        saver.restore(sess, chk_path)
        return True
    
    return False
    

def logistic_regression_fit(features, targets):
    '''
    features: scaled feature including bais terms
    targets: binary target 0 or 1
    '''
    n_epoches = 1000
    lr = 0.1
    ep = 1e-15
    m,n = features.shape
    X = tf.constant(features, dtype=tf.float32, name="X")
    y = tf.constant(targets, dtype=tf.float32, name="y")
    
    theta = tf.Variable(tf.random_uniform((n,1),-1,1), dtype=tf.float32, name="theta")
    cur_epoch = tf.Variable(0, dtype=tf.int32, name="cur_epoch")
    
    with tf.name_scope("loss") as scope:
        lin_preds = tf.matmul(X, theta)
        preds = tf.sigmoid(lin_preds)
        log_loss = -tf.reduce_mean(y*tf.log(preds+ep)+(1-y)*tf.log(1-preds+ep))
#         log_loss = tf.losses.log_loss(y,preds)
    
    with tf.name_scope("train") as scope:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        train = optimizer.minimize(log_loss)
        
    with tf.name_scope("init") as scope:
        init = tf.global_variables_initializer()
    
    with tf.name_scope("save") as scope:
        saver = tf.train.Saver()
        inc_epoch = tf.assign(cur_epoch, cur_epoch+1)

    with ScalerGraphSaver("ch09_excercise", log_loss) as graph_saver:
        with tf.Session() as sess:
            if not restore_checkpoint():
                sess.run(init)

            epoch_start = cur_epoch.eval()
            display(epoch_start)

            for epoch in range(epoch_start, n_epoches):
                sess.run(train)
                sess.run(inc_epoch)
                graph_saver.log_summary(epoch, None)
                if epoch%100==0:
                    save_checkpoint(epoch)
                    print("log_loss: {}".format(log_loss.eval()))
                if epoch==500:
                    raise  Exception("Terminated to test saver")
            best_theta = theta.eval()
    
    return best_theta
    

def logistic_regression_predict(theta,X):
    theta_tf = tf.constant(theta, dtype=tf.float32, name="theta")
    X_tf = tf.constant(X, dtype=tf.float32, name="X")
    lin_preds = tf.matmul(X_tf, theta_tf)
    preds = tf.sigmoid(lin_preds)
    with tf.Session() as sess:
        predictions = preds.eval()
    return predictions>=0.5

/Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [167]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler
import numpy as np
m,n = 3000,5
X,y = make_moons(m, shuffle=True, noise=0.2)
# X = StandardScaler().fit_transform(X)
X = np.c_[np.ones((m,1)), X]


In [272]:
tf.reset_default_graph()
theta = logistic_regression_fit(X,y.reshape(-1,1))
theta

501

log_loss: 0.31353750824928284
log_loss: 0.3081745505332947
log_loss: 0.30435600876808167
log_loss: 0.30157625675201416


array([[ 0.23651123],
       [ 1.2551633 ],
       [-3.552615  ]], dtype=float32)

In [273]:
from sklearn.metrics import accuracy_score
tn = -1
preds = logistic_regression_predict(theta,X[:tn])
display(accuracy_score(y[:tn], preds))
np.c_[y[:tn], preds]

0.8666222074024675

array([[0, 0],
       [1, 1],
       [0, 0],
       ...,
       [1, 1],
       [1, 1],
       [0, 0]])

In [145]:
y

array([1, 1, 1, ..., 0, 1, 1])

In [165]:
tf.train.Saver?

In [ ]:
saver = tf.train.Saver

In [250]:
saver.last_checkpoints?